# Prepare dataset

- Create dataset from the new dataframe

## Librairies

In [1]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 11.5 MB 15.3 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 3.1 MB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 39.8 MB/s ta 0:00:01
     |████████████████████████████████| 3.3 MB 37.6 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 290 kB 17.0 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 16.6 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 21.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 31.6 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 35.8 MB/s eta 0:00:01
     |████████████████████████████████| 29.4 MB 24.1 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 36.1 MB/s eta 0:00:01
     |████████████████

In [2]:
import os
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [3]:
print('pandas:\t\t', pd.__version__)
print('transformers:\t', transformers.__version__)
print('datasets:\t', datasets.__version__)

pandas:		 1.3.4
transformers:	 4.12.5
datasets:	 1.15.1


## Global variables

In [8]:
dataset_df = pd.read_pickle("/notebooks/ywc_token_classification/ywc_dataset_TokCL_pickle_by_speech")

In [9]:
dataset_df

,full_speech,Text,Part,Document,Order,Sentence,Start,End,Annotator,Tag,...,full_speech_words,tags_list,multi_class_tags,bio_only_tags,posfull_tokens_list,full_speech_words_proper,is_equal_tag,mc_int_tags,bio_int_tags,is_equal_tag_four
0,ANDERSON: I also think that that unborn child ...,And for the state to interfere in that decisio...,70,8_1980,2,2,49535,50102,NaN,"{""O"": 4, ""Claim"": 88, ""Premise"": 22}",...,"[ANDERSON, :, I, also, think, that, that, unbo...","[Claim, Claim, Claim, Claim, Claim, Claim, Cla...","[I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-C...","[I-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, I, also, think, that, that, unbo...","[ANDERSON, :, I, also, think, that, that, unbo...",1,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
1,"ANDERSON: Miss Loomis, I think it's very appro...",In other words....,3,8_1980,12,12,5237,5256,NaN,"{""O"": 5}",...,"[ANDERSON, :, Miss, Loomis, ,, I, think, it, '...","[Premise, Premise, Premise, Premise, Premise, ...","[B-PREM, I-PREM, I-PREM, I-PREM, I-PREM, I-PRE...","[B-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, Miss, Loomis, ,, I, think, it, '...","[ANDERSON, :, Miss, Loomis, ,, I, think, it, '...",1,"[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
2,"ANDERSON: Miss Quinn, I would be very happy to...","But I think, by improving productivity, they w...",51,8_1980,18,18,37938,38090,NaN,"{""O"": 2, ""Claim"": 23}",...,"[ANDERSON, :, Miss, Quinn, ,, I, would, be, ve...","[Claim, Claim, Claim, Claim, Claim, Claim, Cla...","[I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-C...","[I-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, Miss, Quinn, ,, I, would, be, ve...","[ANDERSON, :, Miss, Quinn, ,, I, would, be, ve...",1,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
3,"ANDERSON: Mr. Corddry, I am well aware of the ...","But I would leave you with this thought, sir, ...",27,8_1980,14,14,20806,21191,NaN,"{""O"": 32, ""Premise"": 41, ""Claim"": 8}",...,"[ANDERSON, :, Mr., Corddry, ,, I, am, well, aw...","[Claim, Claim, Claim, Claim, Claim, Claim, Cla...","[B-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-C...","[B-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, Mr., Corddry, ,, I, am, well, aw...","[ANDERSON, :, Mr., Corddry, ,, I, am, well, aw...",1,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
4,"ANDERSON: Mr. Greenberg, I think my opponent i...",I think that conservation - I think that a cha...,24,8_1980,6,6,18041,18185,NaN,"{""O"": 8, ""Claim"": 22}",...,"[ANDERSON, :, Mr., Greenberg, ,, I, think, my,...","[Claim, Claim, Claim, Claim, Claim, Claim, Cla...","[I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-C...","[I-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, Mr., Greenberg, ,, I, think, my,...","[ANDERSON, :, Mr., Greenberg, ,, I, think, my,...",1,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,TRUMP: You should have changed the law. But yo...,"I mean, I sat in my apartment today on a very ...",231,43_2016,2,2,31408,31504,NaN,"{""O"": 4, ""Premise"": 17}",...,"[TRUMP, :, You, should, have, changed, the, la...","[O, O, O, O, O, O, O, O, O, Claim, Claim, Clai...","[O, O, O, O, O, O, O, O, O, B-CL, I-CL, I-CL, ...","[O, O, O, O, O, O, O, O, O, B-COMP, I-COMP, I-...","[TRUMP, :, You, should, have, changed, the, la...","[TRUMP, :, You, should, have, changed, the, la...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, ...",1
3067,TRUMP: You were in total contact with the Whit...,I don’t think he would be listening to you ver...,188,42_2016,1,1,27618,27680,NaN,"{""O"": 15}",...,"[TRUM

In [10]:
dataset_df.columns

Index(['full_speech', 'Text', 'Part', 'Document', 'Order', 'Sentence', 'Start',
       'End', 'Annotator', 'Tag', 'Component', 'Speech', 'Speaker',
       'SpeakerType', 'Set', 'Date', 'Year', 'Name', 'MainTag',
       'full_speech_words', 'tags_list', 'multi_class_tags', 'bio_only_tags',
       'posfull_tokens_list', 'full_speech_words_proper', 'is_equal_tag',
       'mc_int_tags', 'bio_int_tags', 'is_equal_tag_four'],
      dtype='object')

In [11]:
# sanity check
print(len(dataset_df))
# dataset_df = dataset_df.dropna()
print(len(dataset_df))

3071
3071


In [12]:
dataset_df = dataset_df.reset_index(drop=True)

In [13]:
dataset_df

,full_speech,Text,Part,Document,Order,Sentence,Start,End,Annotator,Tag,...,full_speech_words,tags_list,multi_class_tags,bio_only_tags,posfull_tokens_list,full_speech_words_proper,is_equal_tag,mc_int_tags,bio_int_tags,is_equal_tag_four
0,ANDERSON: I also think that that unborn child ...,And for the state to interfere in that decisio...,70,8_1980,2,2,49535,50102,NaN,"{""O"": 4, ""Claim"": 88, ""Premise"": 22}",...,"[ANDERSON, :, I, also, think, that, that, unbo...","[Claim, Claim, Claim, Claim, Claim, Claim, Cla...","[I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-C...","[I-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, I, also, think, that, that, unbo...","[ANDERSON, :, I, also, think, that, that, unbo...",1,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
1,"ANDERSON: Miss Loomis, I think it's very appro...",In other words....,3,8_1980,12,12,5237,5256,NaN,"{""O"": 5}",...,"[ANDERSON, :, Miss, Loomis, ,, I, think, it, '...","[Premise, Premise, Premise, Premise, Premise, ...","[B-PREM, I-PREM, I-PREM, I-PREM, I-PREM, I-PRE...","[B-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, Miss, Loomis, ,, I, think, it, '...","[ANDERSON, :, Miss, Loomis, ,, I, think, it, '...",1,"[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
2,"ANDERSON: Miss Quinn, I would be very happy to...","But I think, by improving productivity, they w...",51,8_1980,18,18,37938,38090,NaN,"{""O"": 2, ""Claim"": 23}",...,"[ANDERSON, :, Miss, Quinn, ,, I, would, be, ve...","[Claim, Claim, Claim, Claim, Claim, Claim, Cla...","[I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-C...","[I-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, Miss, Quinn, ,, I, would, be, ve...","[ANDERSON, :, Miss, Quinn, ,, I, would, be, ve...",1,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
3,"ANDERSON: Mr. Corddry, I am well aware of the ...","But I would leave you with this thought, sir, ...",27,8_1980,14,14,20806,21191,NaN,"{""O"": 32, ""Premise"": 41, ""Claim"": 8}",...,"[ANDERSON, :, Mr., Corddry, ,, I, am, well, aw...","[Claim, Claim, Claim, Claim, Claim, Claim, Cla...","[B-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-C...","[B-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, Mr., Corddry, ,, I, am, well, aw...","[ANDERSON, :, Mr., Corddry, ,, I, am, well, aw...",1,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
4,"ANDERSON: Mr. Greenberg, I think my opponent i...",I think that conservation - I think that a cha...,24,8_1980,6,6,18041,18185,NaN,"{""O"": 8, ""Claim"": 22}",...,"[ANDERSON, :, Mr., Greenberg, ,, I, think, my,...","[Claim, Claim, Claim, Claim, Claim, Claim, Cla...","[I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-CL, I-C...","[I-COMP, I-COMP, I-COMP, I-COMP, I-COMP, I-COM...","[ANDERSON, :, Mr., Greenberg, ,, I, think, my,...","[ANDERSON, :, Mr., Greenberg, ,, I, think, my,...",1,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,TRUMP: You should have changed the law. But yo...,"I mean, I sat in my apartment today on a very ...",231,43_2016,2,2,31408,31504,NaN,"{""O"": 4, ""Premise"": 17}",...,"[TRUMP, :, You, should, have, changed, the, la...","[O, O, O, O, O, O, O, O, O, Claim, Claim, Clai...","[O, O, O, O, O, O, O, O, O, B-CL, I-CL, I-CL, ...","[O, O, O, O, O, O, O, O, O, B-COMP, I-COMP, I-...","[TRUMP, :, You, should, have, changed, the, la...","[TRUMP, :, You, should, have, changed, the, la...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, ...",1
3067,TRUMP: You were in total contact with the Whit...,I don’t think he would be listening to you ver...,188,42_2016,1,1,27618,27680,NaN,"{""O"": 15}",...,"[TRUM

In [15]:
dataset_df.Set.value_counts()

TRAIN         1589
TEST           847
VALIDATION     635
Name: Set, dtype: int64

In [17]:
train_df = dataset_df[dataset_df.Set=='TRAIN'].reset_index(drop=True)
test_df = dataset_df[dataset_df.Set=='TEST'].reset_index(drop=True)



# for example

val_df = dataset_df[dataset_df.Set=='VALIDATION'].reset_index(drop=True)

## Prepare Dataset object

In [18]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)
dataset_val = Dataset.from_pandas(val_df)

In [12]:
# train_val_datasets = dataset_train.train_test_split(train_size=0.8)
# dataset_train = train_val_datasets['train']
# dataset_val = train_val_datasets['test']

In [19]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test, "validation": dataset_val})

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['full_speech', 'Text', 'Part', 'Document', 'Order', 'Sentence', 'Start', 'End', 'Annotator', 'Tag', 'Component', 'Speech', 'Speaker', 'SpeakerType', 'Set', 'Date', 'Year', 'Name', 'MainTag', 'full_speech_words', 'tags_list', 'multi_class_tags', 'bio_only_tags', 'posfull_tokens_list', 'full_speech_words_proper', 'is_equal_tag', 'mc_int_tags', 'bio_int_tags', 'is_equal_tag_four'],
        num_rows: 1589
    })
    test: Dataset({
        features: ['full_speech', 'Text', 'Part', 'Document', 'Order', 'Sentence', 'Start', 'End', 'Annotator', 'Tag', 'Component', 'Speech', 'Speaker', 'SpeakerType', 'Set', 'Date', 'Year', 'Name', 'MainTag', 'full_speech_words', 'tags_list', 'multi_class_tags', 'bio_only_tags', 'posfull_tokens_list', 'full_speech_words_proper', 'is_equal_tag', 'mc_int_tags', 'bio_int_tags', 'is_equal_tag_four'],
        num_rows: 847
    })
    validation: Dataset({
        features: ['full_speech', 'Text', 'Part', 'Documen

In [21]:
# sanity check
set(dataset['train']['Set']), set(dataset['test']['Set']), set(dataset['validation']['Set'])

({'TRAIN'}, {'TEST'}, {'VALIDATION'})

## Save dataset

In [22]:
torch.save(dataset, os.path.join("/notebooks/ywc_token_classification", 'ywc_TokCL.pt'))

In [17]:
# with open(os.path.join("/notebooks/token_classification/data/", 'pe_dataset_task_1_bio_only.pt'), "wb") as f:
#     pickle.dump(dataset, f)
# #torch.save(dataset, os.path.join("/notebooks/token_classification/data/", 'pe_dataset_task_1_bio_only.pt'))